In [1]:
import pandas as pd
import os
import time

In [2]:
#reading the database
APP_PATH = "C:/users/ishan"
data = pd.read_excel(
     os.path.join(APP_PATH, "Online Retail.xlsx"),
     engine='openpyxl',
)
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [3]:
#finding recency using invoice date and customer ID
recency = data[['InvoiceDate', 'CustomerID']]

In [4]:
recency.apply(pd.Series.nunique)

InvoiceDate    23260
CustomerID      4372
dtype: int64

In [5]:
recency

,InvoiceDate,CustomerID
0,2010-12-01 08:26:00,17850.0
1,2010-12-01 08:26:00,17850.0
2,2010-12-01 08:26:00,17850.0
3,2010-12-01 08:26:00,17850.0
4,2010-12-01 08:26:00,17850.0
...,...,...
541904,2011-12-09 12:50:00,12680.0
541905,2011-12-09 12:50:00,12680.0
541906,2011-12-09 12:50:00,12680.0
541907,2011-12-09 12:50:00,12680.0


In [6]:
recency['InvoiceDate'] = pd.to_datetime(recency.InvoiceDate)
recent = max(recency['InvoiceDate'])

c:\users\ishan\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [7]:
recency = recency.groupby(['CustomerID']).max()
recency

,InvoiceDate
CustomerID,
12346.0,2011-01-18 10:17:00
12347.0,2011-12-07 15:52:00
12348.0,2011-09-25 13:13:00
12349.0,2011-11-21 09:51:00
12350.0,2011-02-02 16:01:00
...,...
18280.0,2011-03-07 09:52:00
18281.0,2011-06-12 10:53:00
18282.0,2011-12-02 11:43:00


In [9]:
#determine quantitatively which customers are the best ones by examining how recently a customer has purchased
recent_purchase = recent - recency['InvoiceDate']
recent_purchase = pd.DataFrame(recent_purchase)
recent_purchase.head()

,InvoiceDate
CustomerID,
12346.0,325 days 02:33:00
12347.0,1 days 20:58:00
12348.0,74 days 23:37:00
12349.0,18 days 02:59:00
12350.0,309 days 20:49:00


In [10]:
#how often they purchase
frequency = data[['CustomerID', 'InvoiceDate']]
frequency = frequency.groupby(['CustomerID']).count()
frequency.head()

,InvoiceDate
CustomerID,
12346.0,2
12347.0,182
12348.0,31
12349.0,73
12350.0,17


In [11]:
#how much the customer spends
monetary = data[['CustomerID', 'UnitPrice']]
monetary = monetary.groupby(['CustomerID']).sum()
monetary.head()

,UnitPrice
CustomerID,
12346.0,2.08
12347.0,481.21
12348.0,178.71
12349.0,605.10
12350.0,65.30


In [12]:
recency = pd.DataFrame(recent_purchase['InvoiceDate'].astype('timedelta64[D]'))
recency.columns = ['recency']
rfm = pd.concat([recency, frequency, monetary], axis=1)
rfm.columns=['recency', 'frequency', 'monetary']
rfm.head()

,recency,frequency,monetary
CustomerID,,,
12346.0,325.0,2,2.08
12347.0,1.0,182,481.21
12348.0,74.0,31,178.71
12349.0,18.0,73,605.10
12350.0,309.0,17,65.30


In [13]:
rfm.quantile([.25, .5, .75, 1], axis=0)

,recency,frequency,monetary
0.25,16.0,17.0,52.8650
0.50,49.0,42.0,130.0950
0.75,142.0,102.0,302.3325
1.00,373.0,7983.0,41376.3300


In [14]:
RFMscores = rfm.copy()
RFMscores['recency'] = [4 if x <= 22 else x for x in RFMscores['recency']]
RFMscores['recency'] = [3 if 22 < x <= 53 else x for x in RFMscores['recency']]
RFMscores['recency'] = [2 if 53 < x <= 111 else x for x in RFMscores['recency']]
RFMscores['recency'] = [1 if x > 111 else x for x in RFMscores['recency']]
RFMscores['frequency'] = [1 if a < 14 else a for a in RFMscores['frequency']]
RFMscores['frequency'] = [2 if 18 > a >= 14 else a for a in RFMscores['frequency']]
RFMscores['frequency'] = [3 if 22 > a >= 18 else a for a in RFMscores['frequency']]
RFMscores['frequency'] = [4 if a >= 22 else a for a in RFMscores['frequency']]
RFMscores['monetary'] = [1 if x < 781 else x for x in RFMscores['monetary']]
RFMscores['monetary'] = [2 if 781 <= x < 1227 else x for x in RFMscores['monetary']]
RFMscores['monetary'] = [3 if 1227 <= x < 1520 else x for x in RFMscores['monetary']]
RFMscores['monetary'] = [4 if 1520 <= x else x for x in RFMscores['monetary']]

In [15]:
#80% of your business comes from 20% of your customers
RFMscores.apply(pd.Series.nunique)
score = pd.DataFrame((RFMscores['recency'] + RFMscores['frequency'] + RFMscores['monetary'])/3, columns=['AggrScore'])
RFMscores = pd.concat([RFMscores, score], axis = 1)
RFMscores.quantile([.80, 1], axis=0)

,recency,frequency,monetary,AggrScore
0.8,4.0,4.0,1.0,3.0
1.0,4.0,4.0,4.0,4.0


In [16]:
loyalty = RFMscores['AggrScore'].iloc[[x >= 3.333333 for x in RFMscores['AggrScore']]]
loyal = pd.DataFrame(loyalty, columns = ['AggrScore'])
LoyalCustomers = list(loyal.index)
LoyalCustomers

[12352.0,
 12359.0,
 12362.0,
 12415.0,
 12417.0,
 12428.0,
 12433.0,
 12437.0,
 12444.0,
 12451.0,
 12471.0,
 12473.0,
 12474.0,
 12476.0,
 12477.0,
 12536.0,
 12539.0,
 12540.0,
 12567.0,
 12569.0,
 12583.0,
 12584.0,
 12597.0,
 12619.0,
 12621.0,
 12626.0,
 12681.0,
 12682.0,
 12683.0,
 12705.0,
 12709.0,
 12712.0,
 12720.0,
 12744.0,
 12748.0,
 12749.0,
 12757.0,
 12766.0,
 12783.0,
 12841.0,
 12856.0,
 12921.0,
 13001.0,
 13004.0,
 13018.0,
 13050.0,
 13069.0,
 13078.0,
 13081.0,
 13089.0,
 13097.0,
 13098.0,
 13102.0,
 13113.0,
 13137.0,
 13148.0,
 13174.0,
 13230.0,
 13263.0,
 13267.0,
 13269.0,
 13319.0,
 13408.0,
 13418.0,
 13437.0,
 13458.0,
 13468.0,
 13488.0,
 13521.0,
 13523.0,
 13534.0,
 13668.0,
 13694.0,
 13709.0,
 13742.0,
 13755.0,
 13767.0,
 13798.0,
 13810.0,
 13871.0,
 13969.0,
 13985.0,
 14030.0,
 14031.0,
 14049.0,
 14051.0,
 14056.0,
 14071.0,
 14085.0,
 14088.0,
 14096.0,
 14156.0,
 14159.0,
 14180.0,
 14194.0,
 14277.0,
 14298.0,
 14367.0,
 14390.0,
 14395.0,


In [28]:
lost = RFMscores['AggrScore'].iloc[[x <= 1.0 for x in RFMscores['AggrScore']]]
lost_customer = pd.DataFrame(lost, columns = ['AggrScore'])
Lost_Customer = list(lost_customer.index)
Lost_Customer

[12346.0,
 12353.0,
 12355.0,
 12361.0,
 12386.0,
 12401.0,
 12402.0,
 12441.0,
 12450.0,
 12505.0,
 12509.0,
 12547.0,
 12548.0,
 12551.0,
 12565.0,
 12574.0,
 12605.0,
 12622.0,
 12641.0,
 12651.0,
 12665.0,
 12666.0,
 12672.0,
 12686.0,
 12732.0,
 12734.0,
 12736.0,
 12738.0,
 12755.0,
 12770.0,
 12791.0,
 12817.0,
 12821.0,
 12829.0,
 12831.0,
 12837.0,
 12855.0,
 12864.0,
 12870.0,
 12873.0,
 12875.0,
 12881.0,
 12888.0,
 12891.0,
 12897.0,
 12902.0,
 12917.0,
 12929.0,
 12943.0,
 12974.0,
 12977.0,
 13002.0,
 13011.0,
 13033.0,
 13059.0,
 13062.0,
 13070.0,
 13072.0,
 13080.0,
 13101.0,
 13106.0,
 13108.0,
 13120.0,
 13132.0,
 13133.0,
 13135.0,
 13144.0,
 13154.0,
 13161.0,
 13185.0,
 13194.0,
 13216.0,
 13217.0,
 13218.0,
 13223.0,
 13227.0,
 13229.0,
 13261.0,
 13262.0,
 13270.0,
 13295.0,
 13296.0,
 13299.0,
 13302.0,
 13307.0,
 13391.0,
 13420.0,
 13452.0,
 13462.0,
 13482.0,
 13484.0,
 13493.0,
 13508.0,
 13513.0,
 13551.0,
 13581.0,
 13618.0,
 13645.0,
 13663.0,
 13682.0,
